# Week 7 - Machine Learning practical

## Questions for investigation  
  
  We'll be exploring the netflix-rotten-tomatoes-metacritic-imdb.csv dataset we were working with last week.  
    
We know that our client is interested in winning awards, and we've tried analysing the data to find patterns using simple relationships between the variables.   

Now we're going to see if machine learning can come up with any insights.  

1. What factors most affect the number of awards won by a movie?  
2. Of those factors, which carry the most weight?

In [ ]:
# import the modules we will need to use 
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

# machine learning models and utilities 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression



# Utilities

### Generating synthetic datasets

In [ ]:
from sklearn.datasets import make_classification, make_blobs
from matplotlib.colors import ListedColormap

cmap_bold = ListedColormap(['#FFFF00', '#00FF00', '#0000FF','#000000'])

# synthetic dataset for simple regression
from sklearn.datasets import make_regression
plt.figure()
plt.title('Sample regression problem with one input variable')
X_R1, y_R1 = make_regression(n_samples = 100, n_features=1,
                            n_informative=1, bias = 150.0,
                            noise = 30, random_state=0)
plt.scatter(X_R1, y_R1, marker= 'o', s=50)
plt.show()


# synthetic dataset for more complex regression
from sklearn.datasets import make_friedman1
plt.figure()
plt.title('Complex regression problem with one input variable')
X_F1, y_F1 = make_friedman1(n_samples = 100,
                           n_features = 7, random_state=0)

plt.scatter(X_F1[:, 2], y_F1, marker= 'o', s=50)
plt.show()

# synthetic dataset for classification (binary) 
plt.figure()
plt.title('Sample binary classification problem with two informative features')
X_C2, y_C2 = make_classification(n_samples = 100, n_features=2,
                                n_redundant=0, n_informative=2,
                                n_clusters_per_class=1, flip_y = 0.1,
                                class_sep = 0.5, random_state=0)
plt.scatter(X_C2[:, 0], X_C2[:, 1], c=y_C2,
           marker= 'o', s=50, cmap=cmap_bold)
plt.show()


# more difficult synthetic dataset for classification (binary) 
# with classes that are not linearly separable
X_D2, y_D2 = make_blobs(n_samples = 100, n_features = 2, centers = 8,
                       cluster_std = 1.3, random_state = 4)
y_D2 = y_D2 % 2
plt.figure()
plt.title('Sample binary classification problem with non-linearly separable classes')
plt.scatter(X_D2[:,0], X_D2[:,1], c=y_D2,
           marker= 'o', s=50, cmap=cmap_bold)
plt.show()

### Visualising model outputs

In [ ]:
def plot_two_class_knn(X, y, n_neighbors, weights, X_test, y_test):
    from sklearn import neighbors
    import matplotlib.patches as mpatches

    X_mat = X
    y_mat = y

    # Create color maps
    cmap_light = ListedColormap(['#FFFFAA', '#AAFFAA', '#AAAAFF','#EFEFEF'])
    cmap_bold  = ListedColormap(['#FFFF00', '#00FF00', '#0000FF','#000000'])

    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X_mat, y_mat)

    # Plot the decision boundary by assigning a color in the color map
    # to each mesh point.

    mesh_step_size = .01  # step size in the mesh
    plot_symbol_size = 50

    x_min, x_max = X_mat[:, 0].min() - 1, X_mat[:, 0].max() + 1
    y_min, y_max = X_mat[:, 1].min() - 1, X_mat[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size),
                         np.arange(y_min, y_max, mesh_step_size))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')

    # Plot training points
    plt.scatter(X_mat[:, 0], X_mat[:, 1], s=plot_symbol_size, c=y, cmap=cmap_bold, edgecolor = 'black')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

    title = "Neighbors = {}".format(n_neighbors)
    if (X_test is not None):
        train_score = clf.score(X_mat, y_mat)
        test_score  = clf.score(X_test, y_test)
        title = title + "\nTrain score = {:.2f}, Test score = {:.2f}".format(train_score, test_score)

    patch0 = mpatches.Patch(color='#FFFF00', label='class 0')
    patch1 = mpatches.Patch(color='#000000', label='class 1')
    plt.legend(handles=[patch0, patch1])

    plt.xlabel('Feature 0')
    plt.ylabel('Feature 1')
    plt.title(title)

    plt.show()

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_C2, y_C2, random_state=0)

plot_two_class_knn(X_train, y_train, 1, 'uniform', X_test, y_test)
plot_two_class_knn(X_train, y_train, 3, 'uniform', X_test, y_test)
plot_two_class_knn(X_train, y_train, 11, 'uniform', X_test, y_test)

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X_R1, y_R1,
                                                   random_state = 0)
linreg = LinearRegression().fit(X_train, y_train)

print('linear model coeff (w): {}'.format(linreg.coef_)) # weighting/slope
print('linear model intercept (b): {:.3f}'.format(linreg.intercept_)) # intercept
print('R-squared score (training): {:.3f}'.format(linreg.score(X_train, y_train))) # error on train
print('R-squared score (test): {:.3f}'.format(linreg.score(X_test, y_test))) # error on test

### Linear regression: example plot 

In [ ]:
plt.figure(figsize=(5,4))
plt.scatter(X_R1, y_R1, marker= 'o', s=50, alpha=0.8)
plt.plot(X_R1, linreg.coef_ * X_R1 + linreg.intercept_, 'r-')
plt.title('Least-squares linear regression')
plt.xlabel('Feature value (x)')
plt.ylabel('Target value (y)')
plt.show()

# The movies dataset

## Fetching and shaping our data 
  
Different machine learning models need the input data shaped in specific ways.  
In general, they need:
* Numeric values for the variables,  
* A target variable (the 'answer' we're looking for).  

The target variable is the class or category we want to detect, in this case whether or not a movie has won any awards.

### Import the data

In [ ]:
# import dataset 
import pandas as pd 
df = pd.read_csv("https://raw.githubusercontent.com/jargonautical/bsuBootcampCohort4/refs/heads/main/week3/netflix-rotten-tomatoes-metacritic-imdb.csv")


### Inspect the data

In [ ]:
df.columns
#df.head()
#df.sample()
#df.describe()

### Create a class target variable

In [ ]:
# create a new column - if awards won > 0 then 1 else 0
df['winner'] = df['Awards Received'].apply(lambda x: 1 if x > 0 else 0)

### Select the numeric variables we will use 

In [ ]:
# we want to use only numeric values, so we supply a list of these
trim_df = df.select_dtypes(include=['float64', 'int64']) 
trim_df.columns

In [ ]:
trim_df.head()

In [ ]:
trim_df['Awards Received'] = trim_df['Awards Received'].fillna(0) 
trim_df.head()


In [ ]:
#trim_df = trim_df.drop(columns = ['Awards Received'], axis=1)
in_df = trim_df.dropna()

### Defining our model inputs X and y

In [ ]:
X = in_df.drop(columns = ['winner'], axis = 1)
y = in_df['winner']

### The train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

## Supervised learning

### Linear regression

In [ ]:
linreg = LinearRegression().fit(X_train, y_train)

print('linear model intercept: {}'.format(linreg.intercept_))
print('linear model coeff:\n{}'.format(linreg.coef_))
print('R-squared score (training): {:.3f}'.format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'.format(linreg.score(X_test, y_test)))

### Ridge regression

In [ ]:
# ridge regression 'herds' all the features and tries to work out which ones are significant

from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)
linridge = Ridge(alpha=0.1).fit(X_train, y_train)

print('Properties dataset')
print('ridge regression linear model intercept: {}'.format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'.format(linridge.coef_))
print('R-squared score (training): {:.3f}'.format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'.format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'.format(np.sum(linridge.coef_ != 0)))

### Ridge regression with feature normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linridge = Ridge(alpha=10.0).fit(X_train_scaled, y_train)

print('Properties dataset')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

### Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier().fit(X_train, y_train)

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

#### Setting max decision tree depth to help avoid overfitting

In [ ]:
clf2 = DecisionTreeClassifier(max_depth = 3).fit(X_train, y_train)

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf2.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf2.score(X_test, y_test)))

#### Visualizing decision trees

In [ ]:
from sklearn import tree 
# Putting the feature names and class names into variables
fn = X.columns
cn = ['winner', 'loser']
tree.plot_tree(clf,
               feature_names = fn, 
               class_names=cn,
               filled = True);
#fig.savefig('../images/plottreedefault.png')

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(clf2,
               feature_names = fn, 
               class_names=cn,
               filled = True,
               fontsize=8
               );

#### Feature importance

In [ ]:
from adspy_shared_utilities import plot_feature_importances

plt.figure(figsize=(10,4), dpi=80)
plot_feature_importances(clf, fn)
plt.show()

print('Feature importances: {}'.format(clf.feature_importances_))

## Unsupervised learning

### k means clustering

In [ ]:
from sklearn.cluster import KMeans

data = in_df.values
inertias = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,11), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)

In [ ]:
pca_num_components = 2
from sklearn.decomposition import PCA
import seaborn as sns
in_df["kmeans_cluster"] = kmeans.labels_

reduced_data = PCA(n_components=pca_num_components).fit_transform(in_df.iloc[:,1:12])
results = pd.DataFrame(reduced_data,columns=['pca1','pca2'])

sns.scatterplot(x="pca1", y="pca2", hue=in_df['kmeans_cluster'], data=results)
plt.title('K-means Clustering with 2 dimensions')
plt.show()